# Query NOW Corpus for articles


link to [now](https://www.english-corpora.org/now/)


## Setup

In [ ]:
cd ..

In [ ]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# python packages
import time
import random
import pandas as pd
from seleniumrequests import Chrome

In [ ]:
# local imports
from src.scraping import now_query

## Search Term Here

In [ ]:
search = "jim hackett"
search = "daniel martinez-valle"
search = "carlos rodriguez pastor"
search = "kaichi yokoyama"
    

csv_name = "_".join(search.split()) + ".csv"
f"creating: {csv_name}"

## Setup the Selenium Window for Querying

This part requries downloading a driver for selenium. I used Chrome to get this running.


In [ ]:
# Change this to where the chromedriver is in your environment
path_to_chromedriver = './chromedriver'

driver = Chrome(executable_path=path_to_chromedriver)
driver.get('https://www.english-corpora.org/now/')
r = driver.request('POST', "https://www.english-corpora.org/now/x2.asp", data=post_data)
r

## pull the data and write a csv

In [ ]:
post_data = now_query.make_first_post_data(search = search)
initial_query = now_query.make_first_query(search)

full_data = []
all_pages = []
response = now_query.query_request(driver, initial_query)
print('response', response)
pages, df = now_query.parse_query_response(response)
full_data.append(df)
all_pages.append(pages)

total_pages = pages['last_page']
print(f'total_pages: {total_pages}')
last_query = initial_query


max_pages = 15

if total_pages is None:
    total_pages = 1

if total_pages > max_pages:
    print(f"instead of the full {total_pages}, we are only scraping {max_pages}")
    total_pages = max_pages

for i in range(total_pages - 1):
    

    this_query = "/now/" + pages['next_ref']
    print('this:', this_query)
    print('last:', last_query)

    delay = 4 + random.random() * 6
    time.sleep(delay)
    response = now_query.query_request(driver, this_query, last_query)
    pages, df = now_query.parse_query_response(response)

    full_data.append(df)
    all_pages.append(pages)
    
    last_query = this_query

    print(len(df), 'data rows pulled')
    print(len(full_data))
    

full_df = pd.concat(full_data, axis=0)
full_df.to_csv(csv_name)

## Check if it worked

In [ ]:
full_df.describe().T

In [ ]:
pd.read_csv(csv_name)